In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 796.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import nltk
import pandas as pd
from string import punctuation

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# Импорт файла

In [ ]:
import urllib.request

url_transcripts = 'https://drive.google.com/uc?export=download&id=16GyPh7GNhRBpksqxBSfE5U9YFaJxPngL'
file_name = 'transcripts.xlsx'
urllib.request.urlretrieve(url_transcripts, file_name)

('transcripts.xlsx', <http.client.HTTPMessage at 0x7b422d202d70>)

In [ ]:
transcripts_df = pd.read_excel('/content/transcripts.xlsx')

In [ ]:
transcripts_df

,speakerID,fileID,transcript,discourse.type,stimulus,time.point
0,PD-001,PD-001-pers-1-present,"Ну, больше всего меня впечатлил подарок мне на...",pers,present,1
1,PD-002,PD-002-pers-1-present,"Не такой уж обычный традиционный подарок, кото...",pers,present,1
2,PD-004,PD-004-pers-1-present,Я была в начальных классах. Я очень... меня пр...,pers,present,1
3,PD-005,PD-005-pers-1-present,"Я не помню свои подарки. Наверное, самый прекр...",pers,present,1
4,PD-006,PD-006-pers-1-present,"так мне запомнился подарок, когда мой молодой ...",pers,present,1
...,...,...,...,...,...,...
64,PD-076,PD-076-pers-1-present,"Это подарок - собака, на Новый год. Я нескольк...",pers,present,1
65,PD-078,PD-078-pers-1-present,"Так, ну хорошо. У меня, когда нет времени сход...",pers,present,1
66,PD-079,PD-079-pers-1-trip,Однажды с папой мы поехали в Крым и остановили...,pers,trip,1
67,PD-080,PD-080-pers-1-trip,"Я один раз пошла гулять с кошкой, она спрыгнул...",pers,trip,1


In [ ]:
# df = pd.read_excel('/content/control_group.xlsx')

In [ ]:
# df

,speakerID,fileID,transcript,discourse.type,stimulus,time.point
0,PN-01,PN-01-instr-1-table,"Необходимо на пол положить столешницу так, что...",instr,table,1
1,PN-01,PN-01-pers-1-present,Самым запоминающимся подарком был последний по...,pers,present,1
2,PN-01,PN-01-pic-1-sportsman,Человек был очень завистливый. Он увидел друго...,pic,sportsman,1
3,PN-02,PN-02-instr-1-chair,Переверните крышку табурета тыльной стороной в...,instr,chair,1
4,PN-02,PN-02-pers-1-present,"Когда я еще училась в школе, ко мне приехала б...",pers,present,1
...,...,...,...,...,...,...
496,TPN-14,TPN-14-instr-2-chair,Аудио запись четыре. Как собрать табуретку. Бе...,instr,chair,2
497,TPN-14,TPN-14-pers-1-present,"Аудио запись три. Расскажите про лучший, самый...",pers,present,1
498,TPN-14,TPN-14-pers-2-trip,"Аудио запись три. Расскажите про лучшее, самое...",pers,trip,2
499,TPN-14,TPN-14-pic-1-sportsman,"Рассказываю подробную историю, изображенную на...",pic,sportsman,1


# Предобработка

In [ ]:
class Preprocessing:
  def __init__(self, df):
    self.nlp = spacy.load("ru_core_news_sm")
    self.stop_words = stopwords.words('russian')

    self.preprocessed = df

    self.preprocessed['tokens'], self.preprocessed['tokens_without_stops'] = zip(*self.preprocessed['transcript'].apply(self.tokenize))

    self.preprocessed['lemmas'], self.preprocessed['lemmas_without_stops'] = zip(*self.preprocessed['transcript'].apply(self.lemmatize))

  def tokenize(self, text):
    """
    Getting all tokens, except punctuation marks
    Return: list of tokens with stopwords, list of tokens without stopwords
    """
    tokens = word_tokenize(text)
    tokens_w_stops = [i.lower() for i in tokens if (i not in punctuation)]
    tokens_wo_stops = [i.lower() for i in tokens if (i not in punctuation) and (i not in self.stop_words)]
    return tokens_w_stops, tokens_wo_stops

  def lemmatize(self, text):
    """
    Getting lemmas from text with and without stopwords
    Return: list of lemmas with stopwords, list of lemmas without stopwords
    """
    doc = self.nlp(text)
    lemmas = [token.lemma_.lower() for token in doc if (token.text not in punctuation)]
    lemmas_without_stops = [token.lemma_.lower() for token in doc if (token.text not in punctuation) and (token.text not in self.stop_words)]
    return lemmas, lemmas_without_stops

  def get_preprocessed_df(self):
    """
    Getter for preprocessed dataframe
    Return: pd.DataFrame
    """
    return self.preprocessed

In [ ]:
preprocessing_cls = Preprocessing(transcripts_df)
preprocessed_df = preprocessing_cls.get_preprocessed_df()
preprocessed_df

,speakerID,fileID,transcript,discourse.type,stimulus,time.point,tokens,tokens_without_stops,lemmas,lemmas_without_stops
0,PD-001,PD-001-pers-1-present,"Ну, больше всего меня впечатлил подарок мне на...",pers,present,1,"[ну, больше, всего, меня, впечатлил, подарок, ...","[ну, впечатлил, подарок, день, рождения, фотоа...","[ну, больше, всего, меня, впечатлить, подарок,...","[ну, впечатлить, подарок, день, рождение, фото..."
1,PD-002,PD-002-pers-1-present,"Не такой уж обычный традиционный подарок, кото...",pers,present,1,"[не, такой, уж, обычный, традиционный, подарок...","[не, обычный, традиционный, подарок, который, ...","[не, такой, уж, обычный, традиционный, подарок...","[не, обычный, традиционный, подарок, который, ..."
2,PD-004,PD-004-pers-1-present,Я была в начальных классах. Я очень... меня пр...,pers,present,1,"[я, была, в, начальных, классах, я, очень, ......","[я, начальных, классах, я, очень, ..., прям, п...","[я, быть, в, начальный, класс, я, очень, ..., ...","[я, начальный, класс, я, очень, ..., прям, про..."
3,PD-005,PD-005-pers-1-present,"Я не помню свои подарки. Наверное, самый прекр...",pers,present,1,"[я, не, помню, свои, подарки, наверное, самый,...","[я, помню, свои, подарки, наверное, самый, пре...","[я, не, помнить, свой, подарок, наверное, самы...","[я, помнить, свой, подарок, наверное, самый, п..."
4,PD-006,PD-006-pers-1-present,"так мне запомнился подарок, когда мой молодой ...",pers,present,1,"[так, мне, запомнился, подарок, когда, мой, мо...","[запомнился, подарок, молодой, человек, первом...","[так, мне, запомниться, подарок, когда, мой, м...","[запомниться, подарок, молодой, человек, первы..."
...,...,...,...,...,...,...,...,...,...,...
64,PD-076,PD-076-pers-1-present,"Это подарок - собака, на Новый год. Я нескольк...",pers,present,1,"[это, подарок, собака, на, новый, год, я, неск...","[это, подарок, собака, новый, год, я, нескольк...","[это, подарок, собака, на, новый, год, я, неск...","[это, подарок, собака, новый, год, я, нескольк..."
65,PD-078,PD-078-pers-1-present,"Так, ну хорошо. У меня, когда нет времени сход...",pers,present,1,"[так, ну, хорошо, у, меня, когда, нет, времени...","[так, у, времени, сходить, подарком, значит, п...","[так, ну, хороший, у, меня, когда, нет, время,...","[так, у, время, сходить, подарок, значить, при..."
66,PD-079,PD-079-pers-1-trip,Однажды с папой мы поехали в Крым и остановили...,pers,trip,1,"[однажды, с, папой, мы, поехали, в, крым, и, о...","[однажды, папой, поехали, крым, остановились, ...","[однажды, с, папа, мы, поехать, в, крым, и, ос...","[однажды, папа, поехать, крым, остановиться, м..."
67,PD-080,PD-080-pers-1-trip,"Я один раз пошла гулять с кошкой, она спрыгнул...",pers,trip,1,"[я, один, раз, пошла, гулять, с, кошкой, она, ...","[я, пошла, гулять, кошкой, спрыгнула, очень, г...","[я, один, раз, пойти, гулять, с, кошка, она, с...","[я, пойти, гулять, кошка, спрыгнуть, очень, гл..."


In [ ]:
preprocessed_df['tokens'] = preprocessed_df['tokens'].apply(', '.join)
preprocessed_df['tokens_without_stops'] = preprocessed_df['tokens_without_stops'].apply(', '.join)
preprocessed_df['lemmas'] = preprocessed_df['lemmas'].apply(', '.join)
preprocessed_df['lemmas_without_stops'] = preprocessed_df['lemmas_without_stops'].apply(', '.join)

In [ ]:
preprocessed_df

,speakerID,fileID,transcript,discourse.type,stimulus,time.point,tokens,tokens_without_stops,lemmas,lemmas_without_stops
0,PD-001,PD-001-pers-1-present,"Ну, больше всего меня впечатлил подарок мне на...",pers,present,1,"ну, больше, всего, меня, впечатлил, подарок, м...","ну, впечатлил, подарок, день, рождения, фотоап...","ну, больше, всего, меня, впечатлить, подарок, ...","ну, впечатлить, подарок, день, рождение, фотоа..."
1,PD-002,PD-002-pers-1-present,"Не такой уж обычный традиционный подарок, кото...",pers,present,1,"не, такой, уж, обычный, традиционный, подарок,...","не, обычный, традиционный, подарок, который, д...","не, такой, уж, обычный, традиционный, подарок,...","не, обычный, традиционный, подарок, который, д..."
2,PD-004,PD-004-pers-1-present,Я была в начальных классах. Я очень... меня пр...,pers,present,1,"я, была, в, начальных, классах, я, очень, ...,...","я, начальных, классах, я, очень, ..., прям, пр...","я, быть, в, начальный, класс, я, очень, ..., м...","я, начальный, класс, я, очень, ..., прям, проб..."
3,PD-005,PD-005-pers-1-present,"Я не помню свои подарки. Наверное, самый прекр...",pers,present,1,"я, не, помню, свои, подарки, наверное, самый, ...","я, помню, свои, подарки, наверное, самый, прек...","я, не, помнить, свой, подарок, наверное, самый...","я, помнить, свой, подарок, наверное, самый, пр..."
4,PD-006,PD-006-pers-1-present,"так мне запомнился подарок, когда мой молодой ...",pers,present,1,"так, мне, запомнился, подарок, когда, мой, мол...","запомнился, подарок, молодой, человек, первом,...","так, мне, запомниться, подарок, когда, мой, мо...","запомниться, подарок, молодой, человек, первый..."
...,...,...,...,...,...,...,...,...,...,...
64,PD-076,PD-076-pers-1-present,"Это подарок - собака, на Новый год. Я нескольк...",pers,present,1,"это, подарок, собака, на, новый, год, я, неско...","это, подарок, собака, новый, год, я, несколько...","это, подарок, собака, на, новый, год, я, неско...","это, подарок, собака, новый, год, я, несколько..."
65,PD-078,PD-078-pers-1-present,"Так, ну хорошо. У меня, когда нет времени сход...",pers,present,1,"так, ну, хорошо, у, меня, когда, нет, времени,...","так, у, времени, сходить, подарком, значит, пр...","так, ну, хороший, у, меня, когда, нет, время, ...","так, у, время, сходить, подарок, значить, приг..."
66,PD-079,PD-079-pers-1-trip,Однажды с папой мы поехали в Крым и остановили...,pers,trip,1,"однажды, с, папой, мы, поехали, в, крым, и, ос...","однажды, папой, поехали, крым, остановились, м...","однажды, с, папа, мы, поехать, в, крым, и, ост...","однажды, папа, поехать, крым, остановиться, ме..."
67,PD-080,PD-080-pers-1-trip,"Я один раз пошла гулять с кошкой, она спрыгнул...",pers,trip,1,"я, один, раз, пошла, гулять, с, кошкой, она, с...","я, пошла, гулять, кошкой, спрыгнула, очень, гл...","я, один, раз, пойти, гулять, с, кошка, она, сп...","я, пойти, гулять, кошка, спрыгнуть, очень, глу..."


In [ ]:
preprocessed_df.to_excel('pd_group_preprocessed.xlsx')

In [ ]:
url_disorders = 'https://drive.google.com/uc?export=download&id=1S1SX9DLcPwElFGNYtABp_mbz0_5YwlgI'
file_name = 'disorders.xlsx'
urllib.request.urlretrieve(url_disorders, file_name)
disorders_df = pd.read_excel('/content/disorders.xlsx')

In [ ]:
preprocessed_df.columns = ['ID',
 'fileID',
 'transcript',
 'discourse.type',
 'stimulus',
 'time.point',
 'tokens',
 'tokens_without_stops',
 'lemmas',
 'lemmas_without_stops']

In [ ]:
new_df = pd.merge(preprocessed_df, disorders_df.drop(['longitudinal', 'group'], axis=1), on='ID', how='inner')

In [ ]:
new_df

,ID,fileID,transcript,discourse.type,stimulus,time.point,tokens,tokens_without_stops,lemmas,lemmas_without_stops,diagnosis,sex,age,education.level,education.years,depression.symptoms,dep.scale,thought.disorder.symptoms,td.scales
0,PD-001,PD-001-pers-1-present,"Ну, больше всего меня впечатлил подарок мне на...",pers,present,1,"ну, больше, всего, меня, впечатлил, подарок, м...","ну, впечатлил, подарок, день, рождения, фотоап...","ну, больше, всего, меня, впечатлить, подарок, ...","ну, впечатлить, подарок, день, рождение, фотоа...",schizotypal.disorder,female,19.0,secondary,11,1,HDRS,0,SAPS
1,PD-002,PD-002-pers-1-present,"Не такой уж обычный традиционный подарок, кото...",pers,present,1,"не, такой, уж, обычный, традиционный, подарок,...","не, обычный, традиционный, подарок, который, д...","не, такой, уж, обычный, традиционный, подарок,...","не, обычный, традиционный, подарок, который, д...",bipolar.affective.disorder,female,26.0,higher,17,1,HDRS,0,SAPS
2,PD-004,PD-004-pers-1-present,Я была в начальных классах. Я очень... меня пр...,pers,present,1,"я, была, в, начальных, классах, я, очень, ...,...","я, начальных, классах, я, очень, ..., прям, пр...","я, быть, в, начальный, класс, я, очень, ..., м...","я, начальный, класс, я, очень, ..., прям, проб...",borderline.personality.disorder,female,16.0,secondary,9,1,HDRS,0,SAPS
3,PD-005,PD-005-pers-1-present,"Я не помню свои подарки. Наверное, самый прекр...",pers,present,1,"я, не, помню, свои, подарки, наверное, самый, ...","я, помню, свои, подарки, наверное, самый, прек...","я, не, помнить, свой, подарок, наверное, самый...","я, помнить, свой, подарок, наверное, самый, пр...",schizotypal.disorder,female,19.0,secondary,11,0,HDRS,0,SAPS
4,PD-006,PD-006-pers-1-present,"так мне запомнился подарок, когда мой молодой ...",pers,present,1,"так, мне, запомнился, подарок, когда, мой, мол...","запомнился, подарок, молодой, человек, первом,...","так, мне, запомниться, подарок, когда, мой, мо...","запомниться, подарок, молодой, человек, первый...",borderline.personality.disorder,female,22.0,higher.unfinished,15,0,HDRS,0,SAPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,PD-076,PD-076-pers-1-present,"Это подарок - собака, на Новый год. Я нескольк...",pers,present,1,"это, подарок, собака, на, новый, год, я, неско...","это, подарок, собака, новый, год, я, несколько...","это, подарок, собака, на, новый, год, я, неско...","это, подарок, собака, новый, год, я, несколько...",schizotypal.disorder,female,16.0,vocational.unfinished,9,0,HDRS,0,SAPS
65,PD-078,PD-078-pers-1-present,"Так, ну хорошо. У меня, когда нет времени сход...",pers,present,1,"так, ну, хорошо, у, меня, когда, нет, времени,...","так, у, времени, сходить, подарком, значит, пр...","так, ну, хороший, у, меня, когда, нет, время, ...","так, у, время, сходить, подарок, значить, приг...",schizophrenia,female,35.0,higher,15,1,HDRS,0,SAPS
66,PD-079,PD-079-pers-1-trip,Однажды с папой мы поехали в Крым и остановили...,pers,trip,1,"однажды, с, папой, мы, поехали, в, крым, и, ос...","однажды, папой, поехали, крым, остановились, м...","однажды, с, папа, мы, поехать, в, крым, и, ост...","однажды, папа, поехать, крым, остановиться, ме...",schizoaffective.disorder,female,20.0,vocational.unfinished,12,0,HDRS,0,SAPS
67,PD-080,PD-080-pers-1-trip,"Я один раз пошла гулять с кошкой, она спрыгнул...",pers,trip,1,"я, один, раз, пошла, гулять, с, кошкой, она, с...","я, пошла, гулять, кошкой, спрыгнула, очень, гл...","я, один, раз, пойти, гулять, с, кошка, она, сп...","я, пойти, гулять, кошка, спрыгнуть, очень, глу...",schizotypal.disorder,female,21.0,higher.unfinished,12,0,HDRS,0,SAPS


In [ ]:
new_df.to_excel('pd_preprocessed_diagnosis.xlsx', sheet_name='PD')

# Clustering

In [ ]:
class DataExtractionBase:

    def __init__(self, link: str) -> None:
        self.category_types = None

    def get_ids(self, sheet_name: str):
        pass

    def get_column_name(self, category: str):
        pass

    def get_series(self, sheet_name: str, category: str):
        pass


class DataExtractionPDTexts(DataExtractionBase):

    def __init__(self, link: str) -> None:
        super().__init__(link)
        # self.dataset_norm = pd.read_excel(link, sheet_name='healthy')
        self.dataset_pd = pd.read_excel(link, sheet_name='PD')
        self.category_types = ['tokens', 'tokens_without_stops', 'lemmas', 'lemmas_without_stops']

    def get_ids(self, sheet_name: str = 'healthy') -> int:
        """
        Getting ID column
        """
        if sheet_name == 'healthy':
            return self.dataset_norm['speakerID']
        return self.dataset_pd['ID']

    def get_series(self,
                   sheet_name: str,
                   category: str) -> pd.DataFrame:
        """
        Getting one of 8 columns:
          from one of the 2 pages of the dataset
          from one of the 4 categories

        sheet_name: healthy | PD
        category: tokens | tokens_without_stops | lemmas | lemmas_without_stops
        """
        if sheet_name == 'healthy':
            return self.dataset_norm[category]

        return self.dataset_pd[category]

In [ ]:
import gensim
import numpy as np
from itertools import permutations

class ClustersDataBase:

    def __init__(self,
                 extractor: DataExtractionBase,
                 model: gensim.models.fasttext.FastTextKeyedVectors) -> None:
        self.extractor = extractor
        self.model = model
        self.healthy_data = None
        self.impediment_data = None
        self.impediment_type = ''

    def get_df(self, sheet):
        pass

    def add_column(self,
                   sheet_name: str,
                   category: str,
                   clusters: pd.Series) -> None:
        pass

    def save_excel(self, path) -> None:
      """
      Saving data with clusters to an Excel file
      """
      with pd.ExcelWriter(path) as writer:
          # self.healthy_data.to_excel(writer, sheet_name='healthy', index=False)
          self.impediment_data.to_excel(writer, sheet_name=self.impediment_type, index=False)


class ClustersDataPDTexts(ClustersDataBase):

    def __init__(self,
                 extractor: DataExtractionPDTexts,
                 model: gensim.models.fasttext.FastTextKeyedVectors) -> None:
        super().__init__(extractor, model)
        # self.id_healthy = extractor.get_ids('healthy')
        self.id_impediment = extractor.get_ids('PD')
        # self.healthy_data = pd.DataFrame(self.id_healthy)
        self.impediment_data = pd.DataFrame(self.id_impediment)
        self.impediment_type = 'PD'

    def get_df(self, sheet):
        if sheet == 'healthy':
            return self.healthy_data
        return self.aphasia_data

    def add_column(self,
                   sheet_name: str,
                   category: str,
                   clusters: pd.Series) -> None:
        """
        Adding a column with clusters
        """
        if sheet_name == 'healthy':
            self.healthy_data[category] = clusters

        else:
            self.impediment_data[category] = clusters

In [ ]:
class Clusterizer:

    def __init__(self, model: gensim.models.fasttext.FastTextKeyedVectors) -> None:
        self._model = model

    def get_vector(self, word):
        """
        Getting vector depending on the model
        """
        if isinstance(self._model, gensim.models.fasttext.FastTextKeyedVectors):
            return self._model[word]

        return self._model.get_word_vector(word)

    def get_cosine_similarity(self, w1, w2):
        """
        Getting cosine similarity depending on model
        """
        if isinstance(self._model, gensim.models.fasttext.FastTextKeyedVectors):
            return self._model.similarity(w1, w2)

        v1 = self._model.get_word_vector(w1)
        v2 = self._model.get_word_vector(w2)

        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

    def cluster(self, word_sequence: list[str]) -> list[list[str]]:
        """
        An implementation of words clustering algorithm
        for verbal fluency test results,
        from N. Lundin et al., 2022.
        b -- current word
        c -- next word
        d -- next word after the next
        """
        words_by_clusters = []
        cluster = []

        for idx, word in enumerate(word_sequence):
            if word == 'PEOS':
                break

            if word == 'BOS':
                b = word
                c = word_sequence[idx + 1]
                d = word_sequence[idx + 2]
                b_c_sim = self.get_cosine_similarity(b, c)
                c_d_sim = self.get_cosine_similarity(c, d)
                continue

            cluster.append(word)
            a_b_sim = b_c_sim   # S(A,B) equals S(B,C) from previous iteration
            b_c_sim = c_d_sim   # S(B,C) equals S(C,D) from previous iteration
            c_d_sim = self.get_cosine_similarity(word_sequence[idx + 1], word_sequence[idx + 2])

            if a_b_sim > b_c_sim and b_c_sim < c_d_sim:  # a condition of a switch
                words_by_clusters.append(cluster)
                cluster = []

        if cluster:
            words_by_clusters.append(cluster)
        return words_by_clusters

    @staticmethod
    def _custom_similarity(embedding_1, embedding_2):  # с этим что-то надо сделать, оно работает для фасттехта?
        return np.dot(gensim.matutils.unitvec(embedding_1),
                      gensim.matutils.unitvec(embedding_2))

    def davies_bouldin_index(self, cluster_sequence: list[list[str]]) -> float:
        """
        The Davies Bouldin index implementation,
        based on https://scikit-learn.org/stable/modules/clustering.html#davies-bouldin-index
        Si -- the average distance between each point of cluster i
        and the centroid of that cluster – also known as cluster diameter;
        Sj -- the average distance between each point of cluster j
        and the centroid of that cluster – also known as cluster diameter;
        Dij -- the distance between cluster centroids i and j;
        Rij -- similarity between clusters i and j.
        """
        centroids_dict = {}
        for cluster in cluster_sequence:
            centroid = sum(self.get_vector(word) for word in cluster) / len(cluster)
            centroids_dict[tuple(cluster)] = centroid

        Si_values_dict = {}
        for cluster in cluster_sequence:
            cluster_centroid = centroids_dict[tuple(cluster)]
            Si = sum(self._custom_similarity(self.get_vector(word), cluster_centroid)
                    for word in cluster) / len(cluster)
            Si_values_dict[tuple(cluster)] = Si

        Rij_max_values = []
        for cluster_1 in cluster_sequence:
            Rij_values = []
            Si = Si_values_dict[tuple(cluster)]

            for cluster_2 in cluster_sequence:
                if cluster_2 == cluster_1:
                    continue
                Sj = Si_values_dict[tuple(cluster_2)]
                Dij = self._custom_similarity(centroids_dict[tuple(cluster_1)], centroids_dict[tuple(cluster_2)])
                Rij = (Si + Sj) / Dij
                Rij_values.append(Rij)

            if Rij_values:
                Rij_max_values.append(max(Rij_values))

        return sum(Rij_max_values) / len(Rij_max_values) if Rij_max_values else None

    def silhouette_score(self, cluster_sequence: list[list[str]]) -> float:
        """
        The Silhouette score implementation,
        based on https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient
        a -- the mean distance between a sample and all other points in the same class;
        b -- the mean distance between a sample and all other points in the next nearest cluster.
        """
        silhouette_coefs = []

        for idx, cluster in enumerate(cluster_sequence):
            for word_1 in cluster:

                a = sum(self.get_cosine_similarity(word_1, word_2)
                    for word_2 in cluster if word_1 != word_2) / len(cluster)

                if idx != len(cluster_sequence) - 1:
                    b = sum(self.get_cosine_similarity(word_1, word_2)
                    for word_2 in cluster_sequence[idx + 1]) / len(cluster_sequence[idx + 1])
                else:
                    b = sum(self.get_cosine_similarity(word_1, word_2)
                    for word_2 in cluster_sequence[idx - 1]) / len(cluster_sequence[idx - 1])

                s = (b - a) / max(a, b)
                silhouette_coefs.append(s)

        return sum(silhouette_coefs) / len(silhouette_coefs)

    @staticmethod
    def evaluate_clustering(DB_values_page: list[float], silhouette_values: list[float]) -> None:
        """
        The computation of all clustering metrics
        to evaluate given clustering model.
        """
        mean_DB_index_value = sum(DB_values_page) / len(DB_values_page)
        mean_silhouette_score_value = sum(silhouette_values) / len(silhouette_values)
        print('The performance of this clustering algorithm: ')
        print(f'Mean value of Davies Bouldin index: {mean_DB_index_value}')
        print(f'Mean value of Silhouette score: {mean_silhouette_score_value}')

In [ ]:
import gensim
import json


class Vectorizer:
    def __init__(self, model: gensim.models.fasttext.FastTextKeyedVectors) -> None:
        self.model = model
        self._vectors_dictionary = {'BOS': self.get_vector('BOS').tolist(),
                                    'EOS': self.get_vector('EOS').tolist(),
                                    'PEOS': self.get_vector('PEOS').tolist()}

    def get_vector(self, word):
        """
        Getting vectors depending on the model, that is given
        """
        if isinstance(self.model, gensim.models.fasttext.FastTextKeyedVectors):
            return self.model[word]
        return self.model.get_word_vector(word)

    def update_dict(self, words: str) -> None:
        """
        Updating the dictionary during each cell vectorising
        """
        for one_word in words.split(', '):
            if one_word not in self._vectors_dictionary:
                self._vectors_dictionary[one_word] = self.get_vector(one_word).tolist()

    def update_json(self) -> None:
        """
        Updating and saving the json file
        """
        with open("/content/vectors.json", "w") as fp:
            json.dump(self._vectors_dictionary, fp, ensure_ascii=False)

    def get_dictionary(self) -> dict:
        """
        In case we need to get the dictionary
        """
        return self._vectors_dictionary

    @staticmethod
    def get_sequence(words_string: str) -> list[str]:
        """
        Getting a list of tokens + tags of beginning and ending
        BOS -- Beginning of Sentence
        PEOS -- pre-End of Sentence
        EOS -- End of Sentence
        """
        return ['BOS'] + words_string.split(', ') + ['PEOS', 'EOS']

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/213.zip -O model.zip

--2024-09-11 19:46:48--  http://vectors.nlpl.eu/repository/20/213.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.200, 2001:700:112::200
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1485270300 (1.4G) [application/zip]
Saving to: ‘model.zip’

model.zip           100%[===================>]   1.38G  23.0MB/s    in 63s     

2024-09-11 19:47:52 (22.5 MB/s) - ‘model.zip’ saved [1485270300/1485270300]



In [ ]:
!unzip model.zip -d model

Archive:  model.zip
  inflating: model/meta.json         
  inflating: model/model.model       
  inflating: model/model.model.vectors_ngrams.npy  
  inflating: model/model.model.vectors.npy  
  inflating: model/model.model.vectors_vocab.npy  
  inflating: model/README            


In [ ]:
# df = pd.ExcelFile('pd_preprocessed_diagnosis.xlsx').sheet_names
# df

['general_massive',
 'schizotypal.disorder',
 'bipolar.affective.disorder',
 'borderline.personality.disorder',
 'recurrent.depressive.disorder',
 'depressive.episode',
 'schizophrenia',
 'schizoaffective.disorder',
 'personality.disorder']

In [ ]:
import gensim
import pandas as pd

def main():
    # defining classes
    model_path = '/content/model/model.model'
    geowac_model = gensim.models.KeyedVectors.load(model_path)
    extractor = DataExtractionPDTexts('/content/pd_preprocessed_diagnosis.xlsx')
    vectoriser = Vectorizer(geowac_model)
    cluster_saver = ClustersDataPDTexts(extractor, geowac_model)
    clusters_getter = Clusterizer(geowac_model)

    # general principle: clustering one cell at a time
    DB_values_page = []
    silhouette_values_page = []

    # pages = pd.ExcelFile('pd_preprocessed_diagnosis.xlsx').sheet_names
    for page in ['PD']:  ## in future will be ['healthy', 'PD']
        DB_values_lexemes_kind = []
        silhouette_values_lexemes_kind = []

        for category in extractor.category_types:
            sequence_series = extractor.get_series(page, category)  # getting words lists from a column
            clusters_list = []  # a list of lists of clusters for current column

            DB_values_column = []
            silhouette_values_column = []

            for words_string in sequence_series:
                if not isinstance(words_string, str):  # dealing with NaNs or other non-string values
                    clusters_list.append([])
                    continue

                tokens_sequence = vectoriser.get_sequence(words_string)
                # string of words coverted to list with special tags

                cell_clusters = clusters_getter.cluster(tokens_sequence)
                # converting list of words to list of clusters
                clusters_list.append(cell_clusters)

                DB_value = clusters_getter.davies_bouldin_index(cell_clusters)
                # calculating Davies Bouldin index for each cell
                if DB_value:
                    DB_values_column.append(DB_value)

                silhouette_value = clusters_getter.silhouette_score(cell_clusters)
                # calculating Silhouette score for each cell
                silhouette_values_column.append(silhouette_value)

            cluster_saver.add_column(page, category,
                                     pd.Series(clusters_list))
            # adding clusters column in a table

            DB_values_lexemes_kind.extend(DB_values_column)
            silhouette_values_lexemes_kind.extend(silhouette_values_column)

        DB_values_page.extend(DB_values_lexemes_kind)
        silhouette_values_page.extend(silhouette_values_lexemes_kind)

    clusters_getter.evaluate_clustering(DB_values_page, silhouette_values_page)
    cluster_saver.save_excel('pd_group_clusters_dataset.xlsx')

main()

The performance of this clustering algorithm: 
Mean value of Davies Bouldin index: 7.9903342669372845
Mean value of Silhouette score: 0.1826097156656055


# Create groups

In [ ]:
df_clusters = pd.read_excel('/content/pd_group_clusters_dataset.xlsx')
df_clusters

,ID,tokens,tokens_without_stops,lemmas,lemmas_without_stops
0,PD-001,"[['ну', 'больше', 'всего'], ['меня', 'впечатли...","[['ну', 'впечатлил'], ['подарок', 'день', 'рож...","[['ну', 'больше', 'всего'], ['меня', 'впечатли...","[['ну', 'впечатлить'], ['подарок', 'день', 'ро..."
1,PD-002,"[['не', 'такой'], ['уж', 'обычный', 'традицион...","[['не', 'обычный', 'традиционный', 'подарок', ...","[['не', 'такой'], ['уж', 'обычный', 'традицион...","[['не', 'обычный', 'традиционный', 'подарок'],..."
2,PD-004,"[['я', 'была'], ['в', 'начальных', 'классах'],...","[['я', 'начальных', 'классах'], ['я', 'очень']...","[['я', 'быть', 'в'], ['начальный', 'класс'], [...","[['я', 'начальный', 'класс'], ['я', 'очень'], ..."
3,PD-005,"[['я', 'не', 'помню'], ['свои', 'подарки'], ['...","[['я', 'помню'], ['свои', 'подарки'], ['наверн...","[['я', 'не', 'помнить'], ['свой', 'подарок'], ...","[['я', 'помнить'], ['свой', 'подарок'], ['наве..."
4,PD-006,"[['так', 'мне', 'запомнился', 'подарок'], ['ко...","[['запомнился', 'подарок', 'молодой', 'человек...","[['так', 'мне', 'запомниться', 'подарок'], ['к...","[['запомниться', 'подарок'], ['молодой', 'чело..."
...,...,...,...,...,...
64,PD-076,"[['это', 'подарок'], ['собака', 'на', 'новый',...","[['это', 'подарок'], ['собака', 'новый', 'год'...","[['это', 'подарок'], ['собака', 'на', 'новый',...","[['это', 'подарок'], ['собака', 'новый', 'год'..."
65,PD-078,"[['так', 'ну'], ['хорошо', 'у', 'меня'], ['ког...","[['так', 'у'], ['времени', 'сходить', 'подарко...","[['так', 'ну'], ['хороший', 'у', 'меня'], ['ко...","[['так', 'у'], ['время', 'сходить', 'подарок']..."
66,PD-079,"[['однажды', 'с'], ['папой', 'мы'], ['поехали'...","[['однажды', 'папой'], ['поехали', 'крым'], ['...","[['однажды', 'с', 'папа', 'мы'], ['поехать', '...","[['однажды', 'папа', 'поехать', 'крым'], ['ост..."
67,PD-080,"[['я', 'один', 'раз'], ['пошла', 'гулять', 'с'...","[['я', 'пошла'], ['гулять', 'кошкой', 'спрыгну...","[['я', 'один', 'раз'], ['пойти', 'гулять', 'с'...","[['я', 'пойти', 'гулять', 'кошка', 'спрыгнуть'..."


In [ ]:
disorders_df

,ID,longitudinal,group,diagnosis,sex,age,education.level,education.years,depression.symptoms,dep.scale,thought.disorder.symptoms,td.scales
0,PD-001,NaN,patient,schizotypal.disorder,female,19.0,secondary,11,1,HDRS,0,SAPS
1,PD-002,NaN,patient,bipolar.affective.disorder,female,26.0,higher,17,1,HDRS,0,SAPS
2,PD-003,1.0,patient,schizotypal.disorder,female,17.0,secondary,10,0,HDRS,0,SAPS
3,PD-004,1.0,patient,borderline.personality.disorder,female,16.0,secondary,9,1,HDRS,0,SAPS
4,PD-005,1.0,patient,schizotypal.disorder,female,19.0,secondary,11,0,HDRS,0,SAPS
...,...,...,...,...,...,...,...,...,...,...,...,...
341,PN-175,NaN,control,NaN,male,23.0,higher,16,0,QIDS,0,SCL-90-R
342,PN-181,NaN,control,NaN,female,60.0,higher,15,1,QIDS,0,SCL-90-R
343,PN-182,NaN,control,NaN,female,53.0,higher,15,0,QIDS,0,SCL-90-R
344,PN-183,NaN,control,NaN,male,18.0,higher.unfinished,12,1,QIDS,0,SCL-90-R


In [ ]:
class Disorders:
  def __init__(self, disorders_df, clusters_df):
    self.disorders_df = disorders_df.drop(['longitudinal', 'group'], axis=1)
    self.clusters_df = clusters_df
    self.merged_df = pd.merge(clusters_df, disorders_df[['ID', 'diagnosis']], on='ID', how='inner').dropna()

    self.diagnosis = list(self.merged_df['diagnosis'].unique())

  def group_disorders(self, one_diagnosis):
    return self.merged_df[self.merged_df['diagnosis'] == one_diagnosis].drop('diagnosis', axis=1)

  def write_excel(self, path):
    with pd.ExcelWriter(path) as writer:
      self.merged_df.drop('diagnosis', axis=1).to_excel(writer, sheet_name='general_massive')
      for one_diagnosis in self.diagnosis:
        self.group_disorders(one_diagnosis).to_excel(writer, sheet_name=one_diagnosis)

In [ ]:
disorders_connect = Disorders(disorders_df, df_clusters)
disorders_connect.write_excel('pd_clusters_diagnosis.xlsx')